<a href="https://colab.research.google.com/github/safal-tyagi/emotions-recognition/blob/master/emotion_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# load data set from Google drive
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


In [0]:
import pandas as pd
import numpy as np 
posts = pd.read_json("/gdrive/My Drive/nlp_train.json", orient='index')
posts = posts[['body', 'emotion']]
print(posts.head(10))
print(posts.count())

                                                      body                                            emotion
fkrr36o  He was answering a question about the criticis...  {'anger': True, 'anticipation': False, 'disgus...
fjyfp0o  I'm going to start today's discussion thread w...  {'anger': True, 'anticipation': True, 'disgust...
fibm0x7  By announcing the 395 self-quarantined, it pai...  {'anger': True, 'anticipation': True, 'disgust...
fj9b4oj  Likewise, sorry if I offended you. I’m not act...  {'anger': True, 'anticipation': False, 'disgus...
fk04ri5  People infected by experience high fever, coug...  {'anger': False, 'anticipation': False, 'disgu...
flaz17r  &gt;At Philips, major unrest has been caused b...  {'anger': False, 'anticipation': True, 'disgus...
fk3nb9r  I'm not worried about COVID-19 for the same re...  {'anger': False, 'anticipation': True, 'disgus...
fjjwfqp  **32 donation stations across the country**\n\...  {'anger': False, 'anticipation': False, 'disgu...
fhyzjbr  T

In [170]:
# split train test validation data
from sklearn.model_selection import train_test_split

train, test = train_test_split(posts, test_size=0.2)
train, valid = train_test_split(train, test_size=0.2)

print(len(train))
print(len(test))
print(len(valid))

955
299
239


In [0]:
# tokenize text
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(lower=True)
tokenizer.fit_on_texts(posts.body.values)

In [0]:
# save tokenized content
import pickle
file = open("/gdrive/My Drive/tokenizer.pickle", 'wb')
pickle.dump(tokenizer, file)

In [209]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelBinarizer

# max token length for padding
posts_seq = [text.split() for text in posts.body]
tk_posts = tokenizer.texts_to_sequences(posts_seq)
print(len(tk_posts))
print(np.max(np.max(tk_posts)))

# tokenize data
train_seq = [text.split() for text in train.body]
test_seq = [text.split() for text in test.body]
valid_seq = [text.split() for text in valid.body]

# convert seq
tk_train = tokenizer.texts_to_sequences(train_seq)
tk_test = tokenizer.texts_to_sequences(test_seq)
tk_valid = tokenizer.texts_to_sequences(valid_seq)

# pad with max len
x_train = pad_sequences(tk_train, maxlen=500, padding='post')
x_test = pad_sequences(tk_test, maxlen=500, padding='post')
x_valid = pad_sequences(tk_valid, maxlen=500, padding='post')
# sample print
print(np.shape(x_train))
print(x_train[0:2])
print(x_test[0:2])
print(x_valid[0:2])

# emotion labels to binary
y_train = [[np.multiply(val, 1) for val in emotion.values()] for emotion in train.emotion]
y_test = [[np.multiply(val, 1) for val in list(emotion.values())] for emotion in test.emotion]
y_valid = [[np.multiply(val, 1) for val in list(emotion.values())] for emotion in valid.emotion]
y_train = np.array(y_train)
y_test = np.array(y_test)
y_valid = np.array(y_valid)
#sample print
print(np.shape(y_train))
print(y_train[0:2])
print(y_test[0:2])
print(y_valid[0:2])

1493
34189
(955, 500)
[[  181    14  1579    23    16   372     8    30     7  3242     2   647
   1249  6973     8    16    52  3023  2307    11  9882 18520  1445     4
     18    43   294   121     1    74    62   471     7     2   515   115
     66    20   122    20    72   361   584    68    73    13   568 18279
   4727     3   855  3807     3   202    32    12   270   110   397   400
    585 33729   173    20  2155  9882     3 18520  1588  3745     3 18521
      2   382    12     3  1467  2338  4988     7    71  1579    39     8
     12    71   471     2 11223  4988  5838  1445    72     7    12    16
      5    96   161   264    39   182   103  1436  3993     6   138    15
     35     7  5151    70    23    11  1419  1655    71    57    10    67
      1    60   761     2  1909   189   206     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0   

In [0]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense, Dropout
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.optimizers import Adam

In [231]:
input_dim = len(tokenizer.word_index) + 1
# words = tokenizer.word_index.keys()
# word_count = len(words)
print(input_dim)
model = Sequential([
    Embedding(input_dim = input_dim, output_dim = 256, mask_zero=True),
    Bidirectional(LSTM(128, return_sequences=True)),
    Bidirectional(LSTM(64)),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(12)
])

34231


In [232]:
model.compile(loss=CategoricalCrossentropy(from_logits=True),
              optimizer=Adam(),
              metrics=['accuracy'])
model.summary()

Model: "sequential_32"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_33 (Embedding)     (None, None, 256)         8763136   
_________________________________________________________________
bidirectional_33 (Bidirectio (None, None, 256)         394240    
_________________________________________________________________
bidirectional_34 (Bidirectio (None, 128)               164352    
_________________________________________________________________
dense_65 (Dense)             (None, 64)                8256      
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense_66 (Dense)             (None, 12)                780       
Total params: 9,330,764
Trainable params: 9,330,764
Non-trainable params: 0
___________________________________________

In [219]:
model.fit(x_train, y_train, epochs=10, validation_data=(x_valid, y_valid), validation_steps=30)

Epoch 1/10
30/30 [==============================] - 117s 4s/step - loss: 29.3381 - accuracy: 0.2325 - val_loss: 37.7600 - val_accuracy: 0.2803
Epoch 2/10
30/30 [==============================] - 117s 4s/step - loss: 44.0236 - accuracy: 0.2597 - val_loss: 53.9264 - val_accuracy: 0.0502
Epoch 3/10
30/30 [==============================] - 115s 4s/step - loss: 61.5403 - accuracy: 0.2031 - val_loss: 72.6553 - val_accuracy: 0.0502
Epoch 4/10
30/30 [==============================] - 116s 4s/step - loss: 73.8393 - accuracy: 0.2230 - val_loss: 87.6622 - val_accuracy: 0.0502
Epoch 5/10
30/30 [==============================] - 114s 4s/step - loss: 89.5089 - accuracy: 0.1749 - val_loss: 104.4210 - val_accuracy: 0.2803
Epoch 6/10
30/30 [==============================] - 116s 4s/step - loss: 107.6097 - accuracy: 0.2000 - val_loss: 122.9364 - val_accuracy: 0.0502
Epoch 7/10
30/30 [==============================] - 127s 4s/step - loss: 119.2959 - accuracy: 0.1770 - val_loss: 135.8963 - val_accuracy: 0

In [0]:
model.save_weights("/gdrive/My Drive/model_weights.h5")

In [222]:
test_loss, test_acc = model.evaluate(x=x_test, y=y_test)

print('Test Loss: {}'.format(test_loss))
print('Test Accuracy: {}'.format(test_acc))

10/10 [==============================] - 6s 607ms/step - loss: 166.5581 - accuracy: 0.0535
Test Loss: 166.55809020996094
Test Accuracy: 0.05351170524954796


In [229]:
y_pred = model.predict(x_test)
print(np.shape(y_pred))

print(test.emotion[0:2])
print(y_pred[0:2])

(299, 12)
[[3513.4736 3515.8044 3515.8245 3515.4172 3221.7644 3155.619  3510.2332
  3514.678  3508.7786 3200.6213 3420.8086 3200.8333]
 [3513.4626 3515.7898 3515.81   3515.4026 3221.7524 3155.6082 3510.2202
  3514.6646 3508.765  3200.6094 3420.7961 3200.8225]]


In [0]:
print(pred[0:2])

870     anger
4097    anger
Name: label, dtype: object
[['anger: 0.017847503', 'anticipation: 0.0024762717', 'disgust: 0.0153170265', 'fear: 0.0039503183', 'joy: 0.11570063', 'love: 0.0025343532', 'neutral: 0.77372', 'optimism: 0.007678659', 'pessimism: 0.013150303', 'sadness: 0.000678027', 'surprise: 0.039906852', 'trust: 0.0070401705'], ['anger: 0.019174794', 'anticipation: 0.0024942586', 'disgust: 0.015852572', 'fear: 0.0039869486', 'joy: 0.116739854', 'love: 0.0024914858', 'neutral: 0.7706478', 'optimism: 0.0077296635', 'pessimism: 0.0133770695', 'sadness: 0.00069372635', 'surprise: 0.039548703', 'trust: 0.0072642933']]
